This script inserts population data from 2020 census (National Statistics) into townships and counties in PostgreSQL.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
# Create a SQLAlchemy engine
conn_string = "postgresql://lizavabistsevits:@localhost:5432/taiwan" # database connection string
engine = create_engine(conn_string)

In [3]:
pop_townships = pd.read_csv('../data/population/各鄉鎮市區常住人口分布.csv', delimiter='	', encoding='utf-16', thousands=',') # remove commas from figures
pop_townships.head()

,縣市別,鄉鎮別,15-64歲,65歲以上,常住人口數,常住人口數 （複製）,未滿15歲
0,宜蘭縣,三星鄉,20048,4119,26466,26466,2299
1,宜蘭縣,大同鄉,2623,506,3999,3999,870
2,宜蘭縣,五結鄉,27137,6370,39141,39141,5634
3,宜蘭縣,冬山鄉,36063,8753,52046,52046,7230
4,宜蘭縣,壯圍鄉,15553,3631,22172,22172,2988


In [4]:
pop_counties = pd.read_csv('../data/population/各縣市常住人口分布.csv', delimiter='	', encoding='utf-16', thousands=',')
pop_counties.head()

,縣市別,Unnamed: 1
0,宜蘭縣,433319
1,花蓮縣,300406
2,金門縣,67173
3,南投縣,430211
4,屏東縣,755047


In [5]:
# Insert population into townships table
# Some townships have the same name (e.g. East District), so the population data is connected based on both native township and county names
with engine.begin() as connection:
    for index, row in pop_townships.iterrows():
        query = text("""
            UPDATE townships t
            SET total_pop = :pop_value 
            FROM counties c 
            WHERE c.code = t.county 
            AND t.orig_name = :township_name
            AND c.orig_name = :county_name
        """)
        connection.execute(query, {"pop_value": row['常住人口數'], "township_name": row['鄉鎮別'], "county_name": row['縣市別']})

In [6]:
# Insert population into counties table based on native county name
with engine.begin() as connection:
    for index, row in pop_counties.iterrows():
        query = text("""
            UPDATE counties
            SET total_pop = :pop_value 
            WHERE orig_name = :county_name
        """)
        connection.execute(query, {"pop_value": row['Unnamed: 1'], "county_name": row['縣市別']})